# Bring all jsons into one dataframe

In [1]:
import pandas as pd
import numpy as np
import pg8000
import os
import matplotlib.pyplot as plt
%matplotlib inline

import json
import h5py

C:\Users\ramr\AppData\Local\Continuum\anaconda2\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
json_path = "\\\\allen\\programs\\celltypes\\workgroups\\279\\Patch-Seq\\all-metadata-files"

In [9]:
with open(json_path) as data_file:
    json_data = json.load(data_file) 
    print json_data.keys()

IOError: [Errno 13] Permission denied: '\\\\allen\\programs\\celltypes\\workgroups\\279\\Patch-Seq\\all-metadata-files'

In [4]:
print json_data['pipettes']

[{u'status': u'SUCCESS', u'manualRoi': u'dLGNShell', u'recording': {u'timeStart': u'14:27:00 -08:00', u'pipetteR': 5, u'timeWholeCellStart': u'14:28:19 -08:00'}, u'extraction': {u'timeExtractionStart': u'14:33:27 -08:00', u'postPatch': u'nucleus_absent', u'timeExtractionEnd': u'14:34:05 -08:00', u'tubeID': u'P2S4_180212_051_A01', u'retractionPressureApplied': -23, u'timeRetractionEnd': u'14:37:11 -08:00', u'pressureApplied': -23, u'nucleus': u'no', u'extractionNotes': u'Alexa fill visible post experiment.', u'endPipetteR': 1000, u'sampleObservations': u'No Bubbles'}, u'depth': 34, u'approach': {u'sliceHealth': u'2', u'cellHealth': u'3', u'pilotName': u'None', u'creCell': u'Cre-'}, u'autoRoi': u'None'}, {u'status': u'FAILURE', u'manualRoi': u'dLGNShell', u'recording': {u'timeStart': u'14:54:23 -08:00', u'pipetteR': 5.4, u'timeWholeCellStart': u'14:57:40 -08:00'}, u'failureNotes': u'Seal Failed', u'depth': 0, u'approach': {u'sliceHealth': u'2', u'cellHealth': u'2', u'pilotName': u'None',

In [5]:
print json_data['pipettes'][0]['recording']

{u'timeStart': u'14:27:00 -08:00', u'pipetteR': 5, u'timeWholeCellStart': u'14:28:19 -08:00'}


In [6]:
from pandas.io.json import json_normalize

def flatten_attempts(slice_info, form_version):
    """Return flattened slice metadata dataframe.
    
    Parameters
    ----------
    slice_info : dict
        A dictionary of slices with nested pipette attempts.
    
    form_version : string
        A string containing the JEM form version.
        (Pre-version 2 contains IVSCC, PatchSeq and Electroporation arrays)
    Returns
    -------
    attempts_slice_df : pandas dataframe
        A dataframe of all pipette attempts along with all slice metadata.
    """
    
    df = json_normalize(slice_info)
    if form_version >= "2":
        ps_array_name = "pipettes"
    else:
        ps_array_name = "pipettesPatchSeqPilot"
    
    try:
        attempts_df = json_normalize(slice_info[ps_array_name])
        attempts_df["limsSpecName"] = df["limsSpecName"].values[0]
        attempts_df["attempt"] = [p+1 for p in attempts_df.index.values]
        attempts_slice_df = pd.merge(df, attempts_df, how="outer", on="limsSpecName")
        attempts_slice_df.drop(ps_array_name, axis=1, inplace=True)
        return attempts_slice_df
    except IndexError:
        pass

In [7]:
jem_version = json_data["formVersion"]
flat_df = flatten_attempts(json_data, jem_version)

flat_df.head()

,acsfProductionDate,blankFillDate,date,flipped,formVersion,internalFillDate,limsSpecName,rigNumber,rigOperator,sliceQuality,...,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.tubeID,failureNotes,manualRoi,recording.pipetteR,recording.timeStart,recording.timeWholeCellStart,status,attempt
0,2018-02-09,2018-01-26,2018-02-12 14:09:37 -08:00,No,2.0.3,2018-01-12,Scnn1a-Tg2-Cre;Ai14-375563.07.01,4,rustym,Good,...,14:33:27 -08:00,14:37:11 -08:00,P2S4_180212_051_A01,NaN,dLGNShell,5.0,14:27:00 -08:00,14:28:19 -08:00,SUCCESS,1
1,2018-02-09,2018-01-26,2018-02-12 14:09:37 -08:00,No,2.0.3,2018-01-12,Scnn1a-Tg2-Cre;Ai14-375563.07.01,4,rustym,Good,...,NaN,NaN,NaN,Seal Failed,dLGNShell,5.4,14:54:23 -08:00,14:57:40 -08:00,FAILURE,2
2,2018-02-09,2018-01-26,2018-02-12 14:09:37 -08:00,No,2.0.3,2018-01-12,Scnn1a-Tg2-Cre;Ai14-375563.07.01,4,rustym,Good,...,NaN,NaN,NaN,Seal Failed,dLGNShell,5.3,15:13:57 -08:00,15:16:00 -08:00,FAILURE,3


In [10]:
json_list = [ 
    "\\\\allen\\programs\\celltypes\\workgroups\\279\\Patch-Seq\\all-metadata-files\\Scnn1a-Tg2-Cre;Ai14-375563.07.01.PS.json",
    "\\\\allen\\programs\\celltypes\\workgroups\\279\\Patch-Seq\\all-metadata-files\\Chat-IRES-Cre-neo;Ai14-374837.05.02.PS.json",
    "\\\\allen\\programs\\celltypes\\workgroups\\279\\Patch-Seq\\all-metadata-files\\Vip-IRES-Cre;Ai14-373812.03.02.PS.json",
    "\\\\allen\\programs\\celltypes\\workgroups\\279\\Patch-Seq\\all-metadata-files\\Crh-IRES-Cre_ZJH;Sst-IRES-FlpO;Ai65-374736.03.02.PS.json",
    "\\\\allen\\programs\\celltypes\\workgroups\\279\\Patch-Seq\\all-metadata-files\\Oxtr-T2A-Cre;Ai14-374536.03.01.PS.json",
    "\\\\allen\\programs\\celltypes\\workgroups\\279\\Patch-Seq\\all-metadata-files\\Vipr2-IRES2-Cre;Pvalb-T2A-FlpO;Ai65-375686.06.01.PS.json",
    "\\\\allen\\programs\\celltypes\\workgroups\\279\\Patch-Seq\\all-metadata-files\\Pvalb-IRES-Cre;Ai14-375844.05.02.PS.json",
    "\\\\allen\\programs\\celltypes\\workgroups\\279\\Patch-Seq\\all-metadata-files\\Sst-IRES-Cre;Ai14-372545.04.01.PS.json",
    "\\\\allen\\programs\\celltypes\\workgroups\\279\\Patch-Seq\\all-metadata-files\\Sst-IRES-Cre;Ai14-368545.02.02.PS.json",
    "\\\\allen\\programs\\celltypes\\workgroups\\279\\Patch-Seq\\all-metadata-files\\Slc32a1-IRES-Cre;Ai14-327668.05.02.PS.json",
    "\\\\allen\\programs\\celltypes\\workgroups\\279\\Patch-Seq\\all-metadata-files\\Pvalb-IRES-Cre;Ai14-375844.03.02.PS.json",
    "\\\\allen\\programs\\celltypes\\workgroups\\279\\Patch-Seq\\all-metadata-files\\Scnn1a-Tg2-Cre;Ai14-375563.06.01.PS.json"
]

json_df = pd.DataFrame()

for json_path in json_list:
    with open(json_path) as data_file:
        slice_info = json.load(data_file)
        jem_version = slice_info["formVersion"]
        flat_df = flatten_attempts(slice_info, jem_version)
        json_df = pd.concat([json_df, flat_df], axis=0)
        
json_df.head()

,acsfProductionDate,approach.anatomicalLocation,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.pilotName,approach.sliceHealth,attempt,...,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID
0,2018-02-09,NaN,3,NaN,Cre-,NaN,NaN,None,2,1,...,NaN,14:27:00 -08:00,14:28:19 -08:00,4,rustym,NaN,Good,SUCCESS,NaN,NaN
1,2018-02-09,NaN,2,NaN,Cre-,NaN,NaN,None,2,2,...,NaN,14:54:23 -08:00,14:57:40 -08:00,4,rustym,NaN,Good,FAILURE,NaN,NaN
2,2018-02-09,NaN,1,NaN,Cre-,NaN,NaN,None,2,3,...,NaN,15:13:57 -08:00,15:16:00 -08:00,4,rustym,NaN,Good,FAILURE,NaN,NaN
0,2018-02-05,NaN,1,NaN,Cre+,NaN,NaN,None,2,1,...,NaN,16:01:52 -08:00,16:04:22 -08:00,1,ramr,NaN,Good,FAILURE,NaN,NaN
1,2018-02-05,NaN,1,NaN,Cre+,NaN,NaN,None,1,2,...,NaN,16:17:16 -08:00,16:19:54 -08:00,1,ramr,NaN,Good,SUCCESS,"Patch/Cell Unstable,Cell Depolarized,Rheobase ...",NaN
